In [62]:
import torch
import shutil
import os
import cv2
import numpy as np
import yaml
import re
from PIL import Image
from io import BytesIO
import torchvision.transforms as tf
import io

In [63]:
interval_size = 1
cam = "02"
first_frame = 0
source_folder = f"/home/angelika/pixelsplat_fisheye/datasets/kitti360/2013_05_28_drive_0000_sync/image_{cam}/data_rgb"

# Get the list of filenames in the source folder
filenames = sorted(os.listdir(source_folder))
last_frame = 249
print(f"Number of frames: {last_frame}")
available_frames = []
nb_frames = last_frame - first_frame + 1
target_im_folder = f"test_images_cam{cam}_{first_frame}_{last_frame}_overlap_{interval_size}"
target_folder = f"/home/angelika/pixelsplat_fisheye/datasets/kitti360/{target_im_folder}"

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# Iterate over the filenames and copy images to the destination folder
for i, filename in enumerate(filenames):
    if i < nb_frames and i%interval_size==0:
        available_frames.append(i)
        source_path = os.path.join(source_folder, filename)
        filename_dest = filename
        destination_path = os.path.join(target_folder, filename_dest)
        #shutil.copy2(source_path, destination_path)


Number of frames: 249


In [64]:
# Copy camera poses
pose_file = f"/home/angelika/pixelsplat_fisheye/datasets/kitti360/2013_05_28_drive_0000_sync/all_cam{cam[1:]}_to_world.txt"

# Extract poses
cam2world = []

# Read cam2world matrices
for index,line in enumerate(open(pose_file, 'r').readlines()):
    if index % interval_size == 0:
        value = list(map(float, line.strip().split(" ")))
        cam2world.append(np.array(value[1:]))

cam2world = np.array(cam2world)
cam2world = cam2world[first_frame:last_frame+1]
print(cam2world.shape)
# Save only the poses to a .txt file
with open(f'{target_folder}/camera_poses.txt', 'w') as file:
    for i, row in enumerate(cam2world):
        if i % interval_size == 0:
            mapped_index = i // interval_size
            if mapped_index < len(available_frames): 
                line = [str(available_frames[mapped_index])] + [str(x) for x in row]
                line = ' '.join(line) + '\n'
                file.write(line)

(250, 16)


In [65]:
image_dict = {}

image_dict['url'] = ""
image_dict['key'] = f"kitti360_cam{cam}_{first_frame}_{last_frame}"

timestamps = torch.as_tensor(available_frames)
image_dict['timestamps'] = timestamps

In [66]:
cam2world12 = torch.tensor(cam2world[:, :12])
two_zeros = torch.zeros((nb_frames, 2))
intrinsics = torch.zeros((nb_frames, 4))


In [67]:
def readYAMLFile(fileName):
    '''make OpenCV YAML file compatible with python'''
    ret = {}
    skip_lines=1    # Skip the first line which says "%YAML:1.0". Or replace it with "%YAML 1.0"
    with open(fileName) as fin:
        for i in range(skip_lines):
            fin.readline()
        yamlFileOut = fin.read()
        myRe = re.compile(r":([^ ])")   # Add space after ":", if it doesn't exist. Python yaml requirement
        yamlFileOut = myRe.sub(r': \1', yamlFileOut)
        ret = yaml.safe_load(yamlFileOut)
    return ret

intrinsics_file = f'/home/angelika/datasets/kitti_360/calibration/image_{cam}.yaml'
intrinsics_dict = readYAMLFile(intrinsics_file)

intrinsics[:, 0] = torch.tensor(intrinsics_dict['projection_parameters']['gamma1']).repeat(nb_frames)
intrinsics[:, 1] = torch.tensor(intrinsics_dict['projection_parameters']['gamma2']).repeat(nb_frames)
intrinsics[:, 2] = torch.tensor(intrinsics_dict['projection_parameters']['u0']).repeat(nb_frames)
intrinsics[:, 3] = torch.tensor(intrinsics_dict['projection_parameters']['v0']).repeat(nb_frames)


In [68]:
# Camera parameters
poses = torch.cat((intrinsics, two_zeros, cam2world12), dim=1)
poses.shape

image_dict['cameras'] = poses

In [69]:
# Convert images and save them to image_dict

flattened_images = []

for i, filename in enumerate(filenames):
    if i < nb_frames:
        available_frames.append(i)
        source_path = os.path.join(source_folder, filename)
        filename_dest = filename
        destination_path = os.path.join(target_folder, filename_dest)
        image = Image.open(source_path)
        im_resize = image.resize((360, 360))
        im_crop = im_resize.crop((80, 220, 720, 580))
        im_crop.save(destination_path)
        
        # Convert image to bytes
        byte_stream = BytesIO()
        im_crop.save(byte_stream, format='PNG')  # Choose appropriate format, e.g., JPEG, PNG, etc.
        image_bytes = byte_stream.getvalue()
        byte_stream.close()

        # Convert bytes to tensor
        frameTensor = torch.tensor(np.frombuffer(image_bytes, dtype=np.uint8))
        
        flattened_images.append(frameTensor)
        

In [70]:
# Test one image
# byte_stream = BytesIO()
# image.save(byte_stream, format='PNG')  # Choose appropriate format, e.g., JPEG, PNG, etc.
# image_bytes = byte_stream.getvalue()
# byte_stream.close()

# frameTensor = torch.tensor(np.frombuffer(image_bytes, dtype=np.uint8))

# Check if the image is loaded correctly
# image2 = Image.open(BytesIO(frameTensor.numpy().tobytes()))
# image2

In [71]:
# Save the images
image_dict['images'] = flattened_images

# Save the image_dict to a .torch file
data = [image_dict]

image_dict_name = "000000"
torch.save(data, f'{target_folder}/{image_dict_name}.torch')
torch.save(data, f'/home/angelika/pixelsplat_fisheye/datasets/kitti360/test/{image_dict_name}.torch')
